# Operationalizer

In [2]:
import pandas as pd
import os

In [7]:
folder = '/Volumes/Extreme SSD/data/final/silkroad2'
file   = 'items.pickle'

pd.read_pickle(os.path.join(folder, file))

,index,vendor,stime,stime_dt,rating,feedback,item,price,freshness,rtime,rtime_dt,location,area
0,0,sweetpuff,1.402429e+09,2014-06-10 19:32:19,5,didnt receive parcel,"0.1g (One Point) of Pure, fluffy N, N-DMT ! :)",0.055051,2,1.402256e+09,2014-06-08 19:32:19,Australia,Worldwide
1,1,sweetpuff,1.402429e+09,2014-06-10 19:32:19,5,"Good communication, next day delivery. Buds lo...","0.1g (One Point) of Pure, fluffy N, N-DMT ! :)",0.055051,4,1.402083e+09,2014-06-06 19:32:19,Australia,Worldwide
2,2,sweetpuff,1.402429e+09,2014-06-10 19:32:19,1,Never arrived. DO NOT FE WITH THIS SELLER or b...,"0.1g (One Point) of Pure, fluffy N, N-DMT ! :)",0.055051,4,1.402083e+09,2014-06-06 19:32:19,Australia,Worldwide
3,3,sweetpuff,1.402429e+09,2014-06-10 19:32:19,5,great package and product will be back,"0.1g (One Point) of Pure, fluffy N, N-DMT ! :)",0.055051,5,1.401997e+09,2014-06-05 19:32:19,Australia,Worldwide
4,4,sweetpuff,1.402429e+09,2014-06-10 19:32:19,1,Nothing arrived..,"0.1g (One Point) of Pure, fluffy N, N-DMT ! :)",0.055051,5,1.401997e+09,2014-06-05 19:32:19,Australia,Worldwide
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362583,22000285,Heisenberg-meds\n94\n,1.415241e+09,2014-11-06 02:34:00,5,"5/5, trusted vendor! Can safely recommend!","Xanax Upjohn, by Pfizer, 1mg/tab - 20 tabs",0.117716,249,1.393728e+09,2014-03-02 02:34:00,Denmark,Worldwide
1362584,22000286,Heisenberg-meds\n94\n,1.415241e+09,2014-11-06 02:34:00,5,"Just as described, fast sending!","Xanax Upjohn, by Pfizer, 1mg/tab - 20 tabs",0.117716,250,1.393641e+09,2014-03-01 02:34:00,Denmark,Worldwide
1362585,22000287,Heisenberg-meds\n94\n,1.415241e+09,2014-11-06 02:34:00,5,"good service, good stealh, good product 5/5/5....","Xanax Upjohn, by Pfizer, 1mg/tab - 20 tabs",0.117716,251,1.393555e+09,2014-02-28 02:34:00,Denmark,Worldwide
1362586,22001639,NW Nugz\n96\n,1.415250e+09,2014-11-06 04:55:34,5,"Totally awesome, great stealth!","Z Sampler, Two 1/2 Zs you choose, see list",0.888435,0,1.415250e+09,2014-11-06 04:55:34,United States,United States
